importing the dataset to notebook environment

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sunilthite/llm-detect-ai-generated-text-dataset")

print("Path to dataset files:", path)

100%|██████████| 18.6M/18.6M [00:01<00:00, 10.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sunilthite/llm-detect-ai-generated-text-dataset/versions/1


In [3]:
path

'/root/.cache/kagglehub/datasets/sunilthite/llm-detect-ai-generated-text-dataset/versions/1'

In [4]:
import pandas as pd
text_df = pd.read_csv(filepath_or_buffer=path+"/Training_Essay_Data.csv")

In [5]:
text_df[text_df['generated']==0].sample(3)

,text,generated
3780,This is the age of technology everybody is con...,0
8108,I believe that the Facial Action Coding System...,0
18465,Most companies look for specific characteristi...,0
